# Torontonian sampling example

*Author: Brajesh Gupt*

This Jupyter notebook uses the Torontonian Fortran library via Python to generate samples from Torontonians.

## Using the Fortran library

The Torontonian sampling Fortran library can be used either via Fortran, or via the Python interface.

### Interfacing via Fortran

If using the library via Fortran, no external dependencies are required. Simply run
```bash
make fortran
```
in the top level directory. The Fortran modules will be compiled, and the modules stored in the directory `include`. To use the module with your own Fortran, simply include the `use torontonian_samples` at the top of the program, and compile the commands
```bash
gfortran -o program program.f90  /path/to/include/*.o -I/path/to/include/
```

See the file `examples/fortran_example.f90` for an example program that uses the Torontonian sampling library.

### Interfacing via Python

To compile the library for use with Python, `NumPy` is required to be installed. This can be installed via `pip`L
```bash
pip install numpy
```
Then, simply run
```bash
make python
```
in the top level directory to compile the Python library. The library `torontonian_samples.cpython-*-.so` will be created, which can then be imported in Python via `import torontonian_samples`.

## Example usage

Import the library:

In [1]:
import torontonian_samples as tor

Import NumPy and random:

In [2]:
import numpy as np
import random

Import the Strawberry Fields package, to enable us to simulate a Guassian Boson Sampling system. This involves initialising a 10-mode system, squeezing each mode by the same magnitude $s$, and then applying a random interferometer.

In [3]:
import strawberryfields as sf
from strawberryfields.ops import *
from strawberryfields.utils import random_interferometer
from strawberryfields.backends.shared_ops import changebasis

/home/nicolas/anaconda2/envs/covarianceproject/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
l = 10
s = np.arcsinh(1.0)
U = random_interferometer(l)
    
eng, q = sf.Engine(10)
with eng:
    for i in range(l):
        Sgate(s) | q[i]
    Interferometer(U) | q
    
state = eng.run('gaussian')

The resulting vector of means (in this case, the displacement is 0):

In [5]:
C = changebasis(10)
r = C @ state.means()
r

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

The resulting covariance matrix:

In [6]:
cov = C @ state.cov() @ C.T
tormat = cov

Note that we perform a change of basis operation, as the Torontonian sampling library expects the covariance in the form $(x_1, p_1, x_2, p_2, \dots)$.

Now we call `torontonian_samples.generatesample`, a Fortran subroutine, as follows to generate a sample: 

In [7]:
tor.torontonian_samples.generatesample(covmat=tormat,mean=r,seed=random.randint(0,10**6),n_sample=l)

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1], dtype=int64)

Now the above function can be called multiple times in a loop to generate as many samples as your heart desires.

In [8]:
samples=[]
for i in range(20):
    tmp = list(tor.torontonian_samples.generatesample(covmat=tormat,mean=r,seed=random.randint(0,10**6),n_sample=l))
    samples.append(tmp)

In [9]:
samples

[[1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 1, 1, 1, 0, 0, 1],
 [1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 0, 0, 1],
 [1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 0, 1, 1, 0, 1, 1],
 [1, 0, 0, 1, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
 [1, 1, 1, 0, 0, 1, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 1, 0, 1],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 1],
 [0, 1, 1, 1, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 1, 1, 1],
 [1, 0, 0, 0, 0, 0, 1, 1, 1, 0],
 [0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 1, 0, 1, 1, 0, 0]]

## Comparing frequencies with probabilities using the Torontonian function

Create a two mode state using the gaussian backend

In [10]:
import strawberryfields.backends.gaussianbackend.gaussiancircuit as gc

In [11]:
from torontonian import tor as tor_func

In the following cell we generate a two mode squeezed vacuum state, which has perfect number correlation. Because of this one will have always correlated clicks

In [26]:
nmodes = 2 ## Set up number of modes
X = np.block([[0*np.identity(nmodes),np.identity(nmodes)],[np.identity(nmodes),0*np.identity(nmodes)]])
## X = [[0,I],[I,0]]
state = gc.GaussianModes(nmodes,hbar=2) ## A Gaussian state in vacuu,
r1 = np.arcsinh(1.0) ## Amount of squeezing
r2 = -r1
state.squeeze(r1,0,0)
state.squeeze(r2,0,1)
theta = np.pi/4
state.beamsplitter(theta,0,0,1)

We use this state and the torontonian function tor_func to calculate the probabilities (see https://arxiv.org/abs/1807.01639 for more detail)

In [27]:
A = np.round(state.Amat(),14)
Q = np.round(state.qmat(),14)
O = X@A
denom = 1/np.sqrt(np.linalg.det(Q))
p11 = denom*tor_func(O)
rows1 = np.array([0,2])
rows2 = np.array([1,3])
O01 = O[:,rows1][rows1]
O10 = O[:,rows2][rows2]
p01 = denom*tor_func(O01)
p10 = denom*tor_func(O10)
p00 = denom

In [57]:
## Let's check the predicted probabilities add up to one
probs = np.array([p00,p01,p10,p11]).real
print("Probability vector",probs)
print("Sum of probabilities = ",sum(probs))

Probability vector [0.5 0.  0.  0.5]
Sum of probabilities =  1.0000000000000142


In [33]:
# We now generate the x-p covariance matrix used by the Torontonian sampler
cov = state.scovmat()
mean =  np.zeros(4)

In [49]:
# We now generate 
n_sample = 500
samples=[]
for i in range(n_sample):
    tmp = list(tor.torontonian_samples.generatesample(covmat=cov,mean=mean,seed=random.randint(0,10**6),n_sample=nmodes))
    samples.append(tmp)

In [50]:
# This is a list of all the possible events
events = [[0,0],[0,1],[1,0],[1,1]]

In [58]:
# Now we can count their frequencies
freq = np.array([samples.count(event) for event in events])
freq

array([246,   0,   0, 254])

In [60]:
# The probabilities associated with the frequencies are now
print("Relative frequencies:",freq/n_sample)
print("Probabilities:",probs)

Relative frequencies: [0.492 0.    0.    0.508]
Probabilities: [0.5 0.  0.  0.5]


In [ ]:
# which is quite close to expected probabilities from the Torontonian function